# OTRv3 Implementation

**This document is a didatic experiment, do not use this code in production!**

In [7]:
import random
import base64
import hashlib

from Crypto import Random
from Crypto.Cipher import AES

class AESCipher(object):

    def __init__(self, key): 
        self.bs = 32
        self.key = hashlib.sha256(key.encode()).digest()

    def encrypt(self, raw):
        raw = self._pad(raw)
        iv = Random.new().read(AES.block_size)
        cipher = AES.new(self.key, AES.MODE_CBC, iv)
        return base64.b64encode(iv + cipher.encrypt(raw))

    def decrypt(self, enc):
        enc = base64.b64decode(enc)
        iv = enc[:AES.block_size]
        cipher = AES.new(self.key, AES.MODE_CBC, iv)
        return self._unpad(cipher.decrypt(enc[AES.block_size:])).decode('utf-8')

    def _pad(self, s):
        return s + (self.bs - len(s) % self.bs) * chr(self.bs - len(s) % self.bs)

    @staticmethod
    def _unpad(s):
        return s[:-ord(s[len(s)-1:])]

aescipher = AESCipher("test")
ciphertext = aescipher.encrypt("This is the rhythm of the night")
ciphertext

b'2mzWg3A12IzeyM4i5ZJsTDGKSM4FLi5mBhmmrqOlPomE51ADyu/+6fDPtD4DkPDM'

In [6]:
raw = aescipher.decrypt(ciphertext)
raw

'This is the rhythm of the night'

## Authenticated Key Exchange (AKE)

This section outlines the version of the SIGMA protocol used as the AKE. All exponentiations are
done modulo a particular 1536-bit prime, and g is a generator of that group, as indicated in the
detailed description below. Alice and Bob's long-term authentication public keys are pubA and
pubB, respectively.

The general idea is that Alice and Bob do an unauthenticated Diffie-Hellman (D-H) key exchange
to set up an encrypted channel, and then do mutual authentication inside that channel.

Bob will be initiating the AKE with Alice.

**1. Picks a random value r (128 bits)**

In [2]:
r = random.randrange(1 << 127, 1 << 128)
r

237847922369643206304305992767437181019

**2. Picks a random value x (at least 320 bits)**

In [3]:
x = random.randrange(1 << 319)
x

348563867621703145886963627618362249416863880713061855643092647870693161713695441673463454731872